### 1. Load data

In [ ]:
library(tidyverse)
options(repr.plot.width=8, repr.plot.height=6)

unstained <- FALSE # TRUE if samples were not stained, TRUE if samples have been stained

if(unstained){
    summary <- read_csv("./unstained/summary.csv")
}else{
    stained_summary_all <- read_csv("./stained/summary.csv") # load summary data
    stained_summary <- dplyr::filter(stained_summary_all, stained_summary_all$population == "bacteria")
    unstained_summary <- read_csv("./unstained/summary.csv")}

meta <- read_csv("metadata.txt",col_types = cols(date = col_character())) 

### 2. Convert metadata

In [ ]:
meta[1:3,] # print the first few lines to know how to parse metadata

In [ ]:
# add required columns (filename, volume and comments) from metadata
file <- paste0(meta$file,".fcs") # format  sample name to filename (.fcs)
time <- meta$date
lat <- meta$lat
lon <- meta$lon
depth <- meta$depth
replicate <- meta$replicate
volume <- meta$volume
stain <- meta$stain
flag <- meta$flag
comments <- meta$comments

# add required metadata for CMAP
# time <- as.POSIXct(meta$date, format="%d/%b/%y", tz="UTC") 
# lat <- NA
# lon <- NA

# add key information from sample label
# label <- matrix(unlist(list(strsplit(meta$label, split=" "))), ncol=3, byrow=T) 
# treatment <- label[,1]
# timepoint <- label[,2]
# replicate <- label[,3]

# create new metadata
metadata <- tibble(file, time, lat, lon, depth, replicate, volume, stain, flag, comments)

### 3. Merge files
#### a. Merge unstained and unstained summary data

In [ ]:
if(unstained == FALSE){
  summary <- merge(unstained_summary, stained_summary, all=TRUE)
  summary[1:3,]
}

#### b. Add metadata to summary data

In [ ]:
all <- merge(summary, metadata, by="file")
all[1:3,]
all$abundance <- all$count / all$volume

### 4. Calculate abundance
#### a. Check abundance

In [ ]:
all %>%
    dplyr::filter(population != "beads") %>%
    ggplot(aes(abundance, -depth, col=population)) + 
    geom_point() + 
    facet_grid(population ~ lat, scale="free_x") + 
    theme_bw() +
    xlab("Abundance (cells uL-1)") + 
    ylab("Depth (m)")

#### b. Calculate bacteria abundance

In [ ]:
new.all <- all

if(unstained == FALSE){

  pro <- subset(all, population == "prochloro")
  bact <- subset(all, population == "bacteria")

  for (i in 1:nrow(pro)){
    file_number <- regmatches(pro$file[i], regexpr(pattern = "[0-9].*fcs" , text = pro$file[i]))  # removes prefix from the current file so the stained and unstained files will be identical
    matching_file_id <- grep(file_number, bact$file) # find the file in stained samples that matches the file number
    id <- which(all$file == bact$file[matching_file_id] & all$population == "bacteria") # return the index of the file that matches the Pro file numbner
    if(length(id) !=0) new.all$abundance[id] <- all$abundance[id] - pro$abundance[i]
  }
}

new.all[1:3,]

#### c. Check new abundance

In [ ]:
new.all %>%
    dplyr::filter(population != "beads") %>%
    ggplot(aes(abundance, -depth, col=population)) + 
    geom_point() + 
    facet_grid(population ~ lat, scale="free_x") + 
    theme_bw() +
    xlab("Abundance (cells uL-1)") + 
    ylab("Depth (m)")

### 5. Plotting
#### a. Surface abundance

In [ ]:
new.all %>%
    dplyr::filter(population != "beads" & population != "unknown" & depth < 20) %>%
    group_by(lat, population) %>%
    summarize(sd = sd(abundance),
              avg=mean(abundance)) %>%
    ggplot(aes(lat, avg, col=population)) +
    geom_point(size=3) +
    geom_linerange(aes(ymin=avg-sd, ymax=avg+sd)) +
    facet_grid(population ~ ., scale="free_y") + 
    theme_bw() +
    ylab("Abundance (cells uL-1)")

#### b. Abundance depth profile

In [ ]:
new.all %>%
    dplyr::filter(population != "beads" & population != "unknown") %>%
    group_by(lat, depth, population) %>%
    summarize(avg=mean(abundance)) %>%
    ggplot(aes(lat, -depth)) + 
    geom_point(aes(colour=avg), size=4) + 
    viridis::scale_colour_viridis(name="Abundance (cells uL-1)",option ="D") +
    facet_grid(population ~ .) + 
    theme_bw() +
    xlab("Latitude") + 
    ylab("Depth (m)")

#### c. Scatter depth profile

In [ ]:
new.all %>%
    dplyr::filter(population != "beads" & population != "unknown") %>%
    group_by(lat, depth, population) %>%
    summarize(avg=log(mean(scatter))) %>%
    ggplot(aes(lat, -depth)) + 
    geom_point(aes(colour=avg), size=4) + 
    viridis::scale_colour_viridis(name="Log Scatter (normalized to beads)",option ="D") +
    facet_grid(population ~ .) + 
    theme_bw() +
    xlab("Latitude") + 
    ylab("Depth (m)")

#### d. Red fluorescence depth profile

In [ ]:
new.all %>%
    dplyr::filter(population != "beads" & population != "unknown" & population != "bacteria") %>%
    group_by(lat, depth, population) %>%
    summarize(avg=log(mean(red))) %>%
    ggplot(aes(lat, -depth)) + 
    geom_point(aes(colour=avg), size=4) + 
    viridis::scale_colour_viridis(name="Log Red fluorescence (normalized to beads)",option ="D") +
    facet_grid(population ~ .) + 
    theme_bw() +
    xlab("Latitude") + 
    ylab("Depth (m)")

### 6. Save data

In [ ]:
library(FCSplankton)
library(openxlsx)

project <- basename(getwd())
cruise <- "MGL1704" # Cruise ID (ex. KM1906); leave blank if samples were not collected during a cruise

cmap_convert(data = new.all , cruise, project, version = "v1.0")